In [ ]:
import socket
import mss
import cv2
import numpy as np
from PIL import Image, ImageDraw
import pyautogui

# Server configuration
UDP_IP = "0.0.0.0"  # Listen on all available interfaces
UDP_PORT = 3000
BUFFER_SIZE = 65507  # Max UDP packet size

# Screen capture configuration
monitor = {"top": 0, "left": 0, "width": 1920, "height": 1080}
sct = mss.mss()
mouse_icon = Image.open("mouse_icon.png").resize((26, 26), Image.NEAREST)
icon_width, icon_height = mouse_icon.size

# Create UDP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind((UDP_IP, UDP_PORT))

print(f"UDP server listening on {UDP_IP}:{UDP_PORT}")

def capture_and_send_frame(client_address):
    # Capture screen
    sct_img = sct.grab(monitor)
    img = Image.frombytes("RGB", (sct_img.width, sct_img.height), sct_img.rgb)

    # Add mouse cursor
    mouse_x, mouse_y = pyautogui.position()
    icon_position = (mouse_x - icon_width // 2, mouse_y - icon_height // 2)
    img.paste(mouse_icon, icon_position, mouse_icon)

    # Convert to numpy array and encode to JPEG
    img_array = np.array(img)
    img_bgr = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
    _, buffer = cv2.imencode('.jpg', img_bgr, [cv2.IMWRITE_JPEG_QUALITY, 80])
    frame = buffer.tobytes()

    # Send frame in chunks if necessary
    chunk_size = BUFFER_SIZE - 100  # Leave some room for headers
    for i in range(0, len(frame), chunk_size):
        chunk = frame[i:i+chunk_size]
        sock.sendto(chunk, client_address)
    
    # Send end-of-frame marker
    sock.sendto(b'EOF', client_address)

while True:
    data, addr = sock.recvfrom(BUFFER_SIZE)
    if data == b'REQUEST_FRAME':
        capture_and_send_frame(addr)

UDP server listening on 0.0.0.0:3000


In [ ]:
import socket
import mss
import cv2
import numpy as np
from PIL import Image, ImageDraw
import pyautogui

# Client configuration
UDP_IP = "127.0.0.1"  # Change this to the server's IP if it's on a different machine
UDP_PORT = 3000
BUFFER_SIZE = 65507  # Max UDP packet size

# Screen capture configuration
monitor = {"top": 0, "left": 0, "width": 1920, "height": 1080}
sct = mss.mss()

# Create UDP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

print(f"UDP client sending to {UDP_IP}:{UDP_PORT}")

def capture_and_send_frame():
    # Capture screen
    sct_img = sct.grab(monitor)
    img = Image.frombytes("RGB", (sct_img.width, sct_img.height), sct_img.rgb)
    
    # Add mouse cursor
    mouse_x, mouse_y = pyautogui.position()
    draw = ImageDraw.Draw(img)
    draw.ellipse([mouse_x-5, mouse_y-5, mouse_x+5, mouse_y+5], fill='red', outline='white')

    # Convert to numpy array and encode to JPEG
    img_array = np.array(img)
    img_bgr = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
    _, buffer = cv2.imencode('.jpg', img_bgr, [cv2.IMWRITE_JPEG_QUALITY, 80])
    frame = buffer.tobytes()

    # Send frame in chunks
    chunk_size = BUFFER_SIZE - 100  # Leave some room for headers
    for i in range(0, len(frame), chunk_size):
        chunk = frame[i:i+chunk_size]
        sock.sendto(chunk, (UDP_IP, UDP_PORT))
    
    # Send end-of-frame marker
    sock.sendto(b'EOF', (UDP_IP, UDP_PORT))

try:
    while True:
        capture_and_send_frame()
except KeyboardInterrupt:
    print("Stopping screen capture...")
finally:
    sock.close()

In [ ]:
import socket
import mss
import cv2
import numpy as np
from PIL import Image, ImageDraw
import pyautogui

# Client configuration
UDP_IP = "3.222.251.30"  # Your AWS instance's public IP
UDP_PORT = 3000
BUFFER_SIZE = 65507  # Max UDP packet size

# Screen capture configuration
monitor = {"top": 0, "left": 0, "width": 1920, "height": 1080}
sct = mss.mss()

# Create UDP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
print(f"UDP client sending to {UDP_IP}:{UDP_PORT}")

def capture_and_send_frame():
    # Capture screen
    sct_img = sct.grab(monitor)
    img = Image.frombytes("RGB", (sct_img.width, sct_img.height), sct_img.rgb)
    
    # Add mouse cursor
    mouse_x, mouse_y = pyautogui.position()
    draw = ImageDraw.Draw(img)
    draw.ellipse([mouse_x-5, mouse_y-5, mouse_x+5, mouse_y+5], fill='red', outline='white')

    # Convert to numpy array and encode to JPEG
    img_array = np.array(img)
    img_bgr = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
    _, buffer = cv2.imencode('.jpg', img_bgr, [cv2.IMWRITE_JPEG_QUALITY, 80])
    frame = buffer.tobytes()

    # Send frame in chunks
    chunk_size = BUFFER_SIZE - 100  # Leave some room for headers
    for i in range(0, len(frame), chunk_size):
        chunk = frame[i:i+chunk_size]
        sock.sendto(chunk, (UDP_IP, UDP_PORT))
    
    # Send end-of-frame marker
    sock.sendto(b'EOF', (UDP_IP, UDP_PORT))

try:
    while True:
        capture_and_send_frame()
except KeyboardInterrupt:
    print("Stopping screen capture...")
finally:
    sock.close()

UDP client sending to 3.222.251.30:3000
